In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tables import *

In [2]:
date_columns = ['expiration_date', 'registration_init_time']

train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv', index_col=0)
item_data = pd.read_csv('../data/songs.csv')
user_data = pd.read_csv('../data/members.csv', parse_dates=date_columns)

/home/light/miniconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
all_data = pd.concat([train_data, test_data])

all_data = all_data.merge(item_data, on='song_id', how='left')
all_data = all_data.merge(user_data, on='msno', how='left')

In [4]:
enc = LabelEncoder()

for col in [
    'msno', 'song_id', 'source_screen_name', 
    'source_system_tab', 'source_type', 'genre_ids', 
    'artist_name', 'composer', 'lyricist', 'gender'
]:
    all_data[col] = enc.fit_transform(all_data[col].fillna('nan'))
    
for col in ['language', 'city', 'registered_via']:
    all_data[col] = enc.fit_transform(all_data[col].fillna(-2)) 

In [8]:
all_data['time'] = all_data.index / len(all_data)

n = len(train_data)
train_data = all_data[:n]
test_data = all_data[n:]

train_data.to_hdf('../data/train_data.hdf', key='wsdm')
test_data.to_hdf('../data/test_data.hdf', key='wsdm')  